# Capstone project

The goal of this project is to find areas with open venues around a given location at given time of the day

Input:

- US address
- Day of the week
- Time of the day
- Radius

Output:

A map of venues opened at that day and time

Logic:

- User OpenCage service API to map te address to lan/lat coordinates
- Find venues in the area using Foursquare API
- For every venue download its schedule. It's a premium call, so the free account will allow only a limited number
- Check if that venue is going to be open during the requested day/time and set a flag
- Clean up data and leave only the open venues
- Map using Folium


## Input data

Target location. Enter a US address.
Target day. Day of the week 1-7
Target time. "0000"-"2359"
Target radius. In meters

In [155]:
target={}
target['address']="160 Spear St, San Francisco, CA"
target['name']="Office in San Francisco"
target['day']=3
target['time']="0030"
# These values are optional
target['radius']=800 # Keep it walkable, about 1 mile
target['latitude']=None # We'll find out later based on the address
target['longtitude']=None # We'll find out later based on the address


## Imports

In [127]:
import numpy as np
import pandas as pd
import folium 
import requests
from opencage.geocoder import OpenCageGeocode

## Definitions

In [128]:
GEOCODERKEY = 'cce05175e31c445c8f4b02445d1215b6' # Geocoder Key

CLIENT_ID = '415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW' # your Foursquare ID
CLIENT_SECRET = 'A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G' # your Foursquare Secret
VERSION = '20200125' # Foursquare API version

LIMIT=100 # Max numer of returned venues

CATEGORIES=[
    '4d4b7104d754a06370d81259', # Arts & Entertainment
    '4d4b7105d754a06373d81259', # Event
    '4d4b7105d754a06374d81259', # Food
    '4d4b7105d754a06376d81259', # Nightlife spot
]

print('Your credentails:')
print('GEOCODERKEY: ' + GEOCODERKEY)
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
GEOCODERKEY: cce05175e31c445c8f4b02445d1215b6
CLIENT_ID: 415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW
CLIENT_SECRET:A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G


## Libraries

In [156]:
def getNearbyVenuesCategoriesLocation(categories, latitude, longitude, radius=500):
    
    venues_list=[]
    for categoryId in categories:
        #print(categoryId)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            radius, 
            LIMIT,
            categoryId)
        #print(url)   
        # make the GET request
        r = requests.get(url).json()
        #print("r:",r)
        if r["response"]['groups']:
            results=r["response"]['groups'][0]['items']
        else:
            return []
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            categoryId,
            v['venue']['id'],
            False, # Is it open at the target time? just a placeholder column for now, we'll find out later
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['CategoryId', 
                  'VenueId', 
                  'Open', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Main program

In [ ]:
First, we need to convert address into lat/lon. We are going to use free locationIQ.com API

In [157]:
geocoder = OpenCageGeocode(GEOCODERKEY)

results = geocoder.geocode(target['address'])

print(u'%f;%f;%s;%s' % (results[0]['geometry']['lat'], 
                        results[0]['geometry']['lng'],
                        results[0]['components']['country_code'],
                        results[0]['annotations']['timezone']['name']))

target['latitude']=results[0]['geometry']['lat'] 
target['longtitude']=results[0]['geometry']['lng'] 

print(target)

37.791512;-122.393649;us;America/Los_Angeles
{'address': '160 Spear St, San Francisco, CA', 'name': 'Office in San Francisco', 'day': 3, 'time': '0030', 'radius': 800, 'latitude': 37.791512, 'longtitude': -122.393649}


Let's pull all venues in the area. For now we assume the are all closed.

In [158]:
target_venues = getNearbyVenuesCategoriesLocation(
                                categories=CATEGORIES,   
                                latitude=target['latitude'],
                                longitude=target['longtitude'],
                                radius=target['radius']
                                  )

In [159]:
target_venues.head(10)

,CategoryId,VenueId,Open,Venue,Venue Latitude,Venue Longitude,Venue Category
0,4d4b7104d754a06370d81259,4ae1f187f964a520048921e3,False,Cupid's Span,37.791541,-122.390013,Outdoor Sculpture
1,4d4b7104d754a06370d81259,4abfec21f964a520fe9220e3,False,Autodesk Gallery,37.794341,-122.394928,Museum
2,4d4b7104d754a06370d81259,4bbc1556e5b0d13a98f16e7c,False,Rincon Annex Historic Post Office Lobby,37.792774,-122.393446,Art Gallery
3,4d4b7104d754a06370d81259,512fe040e4b097f2b80ece7c,False,Pier 24 Photography,37.789281,-122.387695,Art Gallery
4,4d4b7104d754a06370d81259,4ac3ba2af964a520929c20e3,False,San Francisco Railway Museum,37.793975,-122.393860,History Museum
5,4d4b7104d754a06370d81259,40c10d00f964a520df001fe3,False,Embarcadero Center Cinema,37.794871,-122.399648,Indie Movie Theater
6,4d4b7104d754a06370d81259,3fd66200f964a5207bec1ee3,False,111 Minna Gallery,37.787488,-122.399274,Art Gallery
7,4d4b7104d754a06370d81259,49f168eef964a520aa691fe3,False,Punch Line Comedy Club,37.795218,-122.399957,Comedy Club
8,4d4b7104d754a06370d81259,43d28d6ef964a520172e1fe3,False,The Gateway Theatre,37.796725,-122.399840,Indie Theater
9,4d4b7104d754a06370d81259,4ab16bb8f964a520676920e3,False,Variety Screening Room,37.789459,-122.401676,Movie Theater


In [160]:
target_venues.groupby('Venue Category').count()

,CategoryId,VenueId,Open,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
American Restaurant,4,4,4,4,4,4
Amphitheater,1,1,1,1,1,1
Argentinian Restaurant,1,1,1,1,1,1
Art Gallery,9,9,9,9,9,9
Arts & Entertainment,1,1,1,1,1,1
...,...,...,...,...,...,...
Vegetarian / Vegan Restaurant,1,1,1,1,1,1
Vietnamese Restaurant,4,4,4,4,4,4
Whisky Bar,1,1,1,1,1,1


In [161]:
target_venues['VenueId'].count()

199

Ok, now we need to pull hours for every venue and set the open Flag based on the schedule

Unfortunately, the response is a bit fuzzy. The documentation is available at https://developer.foursquare.com/docs/api/venues/hours .

There are two kinds of schedules:

hours: as I understood, this is when the venue is supposed to be open.
popular: this is when people actually go there.

We can get both values returned, one of them or none. So we need to define priorities. For the purpose of this project, the logic is going to be:

if target day/time in 'popular' OR 'hours' - mark venue Open=True

That means if there are conflicting data - we select Open.
If both 'popular' and 'hours' are not present - that would keep the value of Open set to False (the default).

A function to check if a given day/time falls into open window of a response:

In [162]:

def isOpen(tr_1,day,time):
    """
    Checking the schedule in the request req against the day and the time
    return True if either hours or popular include that day and time, otherwise False
    Foursquare uses the "+0100" to indicate the venue stays open after midnight
    """
    print(day,time,tr_1)
    
    curr_day = day
    prev_day = day-1 if day>1 else 7
    
    # Checking the previous day:
    if tr_1['response']['popular']:
        for i in tr_1['response']['popular']['timeframes']:
            print(i['days'])
            if prev_day in i['days']:
                print(i)
                for o in i['open']:
                    print("open: ",o)
                    if o['end'].startswith('+'):
                        if int(time)<int(o['end'][1:]):
                            print("Match 1!")
                            return True
    # Checking the current day:
    if tr_1['response']['popular']:
        for i in tr_1['response']['popular']['timeframes']:
            print(i['days'])
            if curr_day in i['days']:
                print(i)
                for o in i['open']:
                    print("open: ",o)
                    if int(o['start'])<=int(time):
                        if int(time)<int( '2359' if o['end'].startswith('+') else o['end']):
                            print("Match 2!")
                            return True
    
    # Checking the previous day:
    if tr_1['response']['hours']:
        for i in tr_1['response']['hours']['timeframes']:
            print(i['days'])
            if prev_day in i['days']:
                print(i)
                for o in i['open']:
                    print("open: ",o)
                    if o['end'].startswith('+'):
                        if int(time)<int(o['end'][1:]):
                            print("Match 3!")
                            return True
                        
    # Checking the current day:
    if tr_1['response']['hours']:
        for i in tr_1['response']['hours']['timeframes']:
            print(i['days'])
            if curr_day in i['days']:
                print(i)
                for o in i['open']:
                    print("open: ",o)
                    if int(o['start'])<=int(time):
                        if int(time)<int( '2359' if o['end'].startswith('+') else o['end']):
                            print("Match 4!")
                            return True
    print("No Match")
    return False
    
#isOpen(tr_1,target['day'],target['time'])
#isOpen(tr_1,t_day,t_time)

Tests of isOpen:

In [142]:
tr_1= {'meta': 
       {'code': 200, 'requestId': '5e2cddb1c546f3001b7f5085'}, 
       'response': {
           'hours': {}, 
           'popular': {'timeframes': [
               {'days': [6], 'includesToday': True, 'open': [{'start': '1400', 'end': '1500'}, {'start': '1900', 'end': '+0200'}], 'segments': []}, 
               {'days': [7], 'open': [{'start': '1400', 'end': '1800'}, {'start': '2100', 'end': '+0000'}], 'segments': []}, 
               {'days': [1], 'open': [{'start': '0700', 'end': '0800'}, {'start': '1700', 'end': '+0200'}], 'segments': []}, 
               {'days': [2], 'open': [{'start': '0700', 'end': '0800'}, {'start': '1800', 'end': '+0200'}], 'segments': []}, 
               {'days': [3], 'open': [{'start': '0700', 'end': '0800'}, {'start': '1700', 'end': '+0100'}], 'segments': []}, 
               {'days': [4], 'open': [{'start': '0700', 'end': '0800'}, {'start': '1200', 'end': '1300'}, {'start': '1700', 'end': '+0100'}], 'segments': []}, 
               {'days': [5], 'open': [{'start': '0700', 'end': '0800'}, {'start': '1700', 'end': '+0200'}], 'segments': []}]}}}
t_day=3
t_time='0600'

#isOpen(tr_1,target['day'],target['time'])
#isOpen(tr_1,t_day,t_time)

In [171]:
def setOpenFlagForVenues(venues,day,time):
    for venue_id in venues['VenueId']:
        if venue_id:
            print(venue_id)
            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/{}/hours?&client_id={}&client_secret={}&v={}'.format(
                venue_id,
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION)
            print(url)   
            # make the GET request
            resp = requests.get(url)
            if resp.status_code == 200:
                r=resp.json()
                print('Got 200')
                print("r:",r)
                print("hours: ",r['response']['hours'])
                print("popular: ",r['response']['popular'])
                print(venue_id)
                venues.loc[venues['VenueId'] == venue_id, ['Open']] = isOpen(r,day,time)
            else:
                print('Bad response!',resp.status_code)
            #print(venues['VenueID':venue_id])
            #venues.loc[venues['VenueId'] == venue_id, ['Open']] = True #isOpen(r,day,time)
            #venues.loc[venues['VenueId'] == venue_id, ['Open']] = isOpen(r,day,time)
            #venues.loc['Open','venue_id']=isOpen(r,day,time)
    #r=tr_1
    #venue_id='5402a13f498e6650fc99ef2d'
    #print(r)
    #print(isOpen(r,day,time))
    #venues.loc[venues['VenueId'] == venue_id, ['Open']] = True #isOpen(r,day,time)
    #print(venues)

    return

In [169]:
target_venues.head(15)

,CategoryId,VenueId,Open,Venue,Venue Latitude,Venue Longitude,Venue Category
0,4d4b7104d754a06370d81259,4ae1f187f964a520048921e3,False,Cupid's Span,37.791541,-122.390013,Outdoor Sculpture
1,4d4b7104d754a06370d81259,4abfec21f964a520fe9220e3,False,Autodesk Gallery,37.794341,-122.394928,Museum
2,4d4b7104d754a06370d81259,4bbc1556e5b0d13a98f16e7c,False,Rincon Annex Historic Post Office Lobby,37.792774,-122.393446,Art Gallery
3,4d4b7104d754a06370d81259,512fe040e4b097f2b80ece7c,False,Pier 24 Photography,37.789281,-122.387695,Art Gallery
4,4d4b7104d754a06370d81259,4ac3ba2af964a520929c20e3,False,San Francisco Railway Museum,37.793975,-122.393860,History Museum
5,4d4b7104d754a06370d81259,40c10d00f964a520df001fe3,False,Embarcadero Center Cinema,37.794871,-122.399648,Indie Movie Theater
6,4d4b7104d754a06370d81259,3fd66200f964a5207bec1ee3,False,111 Minna Gallery,37.787488,-122.399274,Art Gallery
7,4d4b7104d754a06370d81259,49f168eef964a520aa691fe3,False,Punch Line Comedy Club,37.795218,-122.399957,Comedy Club
8,4d4b7104d754a06370d81259,43d28d6ef964a520172e1fe3,False,The Gateway Theatre,37.796725,-122.399840,Indie Theater
9,4d4b7104d754a06370d81259,4ab16bb8f964a520676920e3,False,Variety Screening Room,37.789459,-122.401676,Movie Theater


In [172]:
setOpenFlagForVenues(target_venues,target['day'],target['time'])

4ae1f187f964a520048921e3
https://api.foursquare.com/v2/venues/4ae1f187f964a520048921e3/hours?&client_id=415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW&client_secret=A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G&v=20200125
Bad response! 429
4abfec21f964a520fe9220e3
https://api.foursquare.com/v2/venues/4abfec21f964a520fe9220e3/hours?&client_id=415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW&client_secret=A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G&v=20200125
Bad response! 429
4bbc1556e5b0d13a98f16e7c
https://api.foursquare.com/v2/venues/4bbc1556e5b0d13a98f16e7c/hours?&client_id=415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW&client_secret=A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G&v=20200125
Bad response! 429
512fe040e4b097f2b80ece7c
https://api.foursquare.com/v2/venues/512fe040e4b097f2b80ece7c/hours?&client_id=415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW&client_secret=A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G&v=20200125
Bad response! 429
4ac3ba2af964a520929c

Bad response! 429
4aebd409f964a520e9c421e3
https://api.foursquare.com/v2/venues/4aebd409f964a520e9c421e3/hours?&client_id=415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW&client_secret=A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G&v=20200125
Bad response! 429
3fd66200f964a52052ed1ee3
https://api.foursquare.com/v2/venues/3fd66200f964a52052ed1ee3/hours?&client_id=415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW&client_secret=A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G&v=20200125
Bad response! 429
56708d9a498ec28d41077ce2
https://api.foursquare.com/v2/venues/56708d9a498ec28d41077ce2/hours?&client_id=415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW&client_secret=A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G&v=20200125
Bad response! 429
4479910ef964a520d8331fe3
https://api.foursquare.com/v2/venues/4479910ef964a520d8331fe3/hours?&client_id=415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW&client_secret=A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G&v=20200125
Bad response! 429
52

Bad response! 429
58e009dc5d6ec64f3b235c34
https://api.foursquare.com/v2/venues/58e009dc5d6ec64f3b235c34/hours?&client_id=415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW&client_secret=A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G&v=20200125
Bad response! 429
459b7818f964a52089401fe3
https://api.foursquare.com/v2/venues/459b7818f964a52089401fe3/hours?&client_id=415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW&client_secret=A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G&v=20200125
Bad response! 429
56098c7a498e84bc593495a4
https://api.foursquare.com/v2/venues/56098c7a498e84bc593495a4/hours?&client_id=415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW&client_secret=A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G&v=20200125
Bad response! 429
53fb8a0f498e6491055ac177
https://api.foursquare.com/v2/venues/53fb8a0f498e6491055ac177/hours?&client_id=415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW&client_secret=A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G&v=20200125
Bad response! 429
58

Bad response! 429
522e29d611d203fd9fc1229b
https://api.foursquare.com/v2/venues/522e29d611d203fd9fc1229b/hours?&client_id=415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW&client_secret=A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G&v=20200125
Bad response! 429
561fd696498e6e0ed70e175d
https://api.foursquare.com/v2/venues/561fd696498e6e0ed70e175d/hours?&client_id=415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW&client_secret=A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G&v=20200125
Bad response! 429
4570691ff964a5205c3e1fe3
https://api.foursquare.com/v2/venues/4570691ff964a5205c3e1fe3/hours?&client_id=415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW&client_secret=A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G&v=20200125
Bad response! 429
53192adc498e20ab5bf6d887
https://api.foursquare.com/v2/venues/53192adc498e20ab5bf6d887/hours?&client_id=415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW&client_secret=A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G&v=20200125
Bad response! 429
55

Bad response! 429
53221a72498e79092136b979
https://api.foursquare.com/v2/venues/53221a72498e79092136b979/hours?&client_id=415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW&client_secret=A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G&v=20200125
Bad response! 429
44b2a62df964a52042351fe3
https://api.foursquare.com/v2/venues/44b2a62df964a52042351fe3/hours?&client_id=415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW&client_secret=A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G&v=20200125
Bad response! 429
4884e11ff964a5203e511fe3
https://api.foursquare.com/v2/venues/4884e11ff964a5203e511fe3/hours?&client_id=415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW&client_secret=A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G&v=20200125
Bad response! 429
3fd66200f964a52018ed1ee3
https://api.foursquare.com/v2/venues/3fd66200f964a52018ed1ee3/hours?&client_id=415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW&client_secret=A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G&v=20200125
Bad response! 429
56

Bad response! 429
49d17c67f964a5208a5b1fe3
https://api.foursquare.com/v2/venues/49d17c67f964a5208a5b1fe3/hours?&client_id=415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW&client_secret=A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G&v=20200125
Bad response! 429
54ac9189498e4a4e79a0e6d4
https://api.foursquare.com/v2/venues/54ac9189498e4a4e79a0e6d4/hours?&client_id=415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW&client_secret=A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G&v=20200125
Bad response! 429
5816530e38faf800501b4391
https://api.foursquare.com/v2/venues/5816530e38faf800501b4391/hours?&client_id=415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW&client_secret=A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G&v=20200125
Bad response! 429
56034044498e268c84b027c8
https://api.foursquare.com/v2/venues/56034044498e268c84b027c8/hours?&client_id=415S5Q2EHALWEGPR0GSJFIB2S0N1WEGJHXH4BV02SNAEOZQW&client_secret=A1LDQGSXYSC0EGYS2F3MOSAFRIFQZ0KJ3DEIOPLBBCSYDQ1G&v=20200125
Bad response! 429
52

In [173]:
target_venues.groupby('Open').count()

,CategoryId,VenueId,Venue,Venue Latitude,Venue Longitude,Venue Category
Open,,,,,,
False,199,199,199,199,199,199


Ok, at this point target_venues have flag 'Open' set to False or True. We want to leave only the Open ones

In [123]:
target_venues.drop( target_venues[ target_venues['Open'] == False ].index , inplace=True)

In [174]:
target_venues.head(10)

,CategoryId,VenueId,Open,Venue,Venue Latitude,Venue Longitude,Venue Category
0,4d4b7104d754a06370d81259,4ae1f187f964a520048921e3,False,Cupid's Span,37.791541,-122.390013,Outdoor Sculpture
1,4d4b7104d754a06370d81259,4abfec21f964a520fe9220e3,False,Autodesk Gallery,37.794341,-122.394928,Museum
2,4d4b7104d754a06370d81259,4bbc1556e5b0d13a98f16e7c,False,Rincon Annex Historic Post Office Lobby,37.792774,-122.393446,Art Gallery
3,4d4b7104d754a06370d81259,512fe040e4b097f2b80ece7c,False,Pier 24 Photography,37.789281,-122.387695,Art Gallery
4,4d4b7104d754a06370d81259,4ac3ba2af964a520929c20e3,False,San Francisco Railway Museum,37.793975,-122.393860,History Museum
5,4d4b7104d754a06370d81259,40c10d00f964a520df001fe3,False,Embarcadero Center Cinema,37.794871,-122.399648,Indie Movie Theater
6,4d4b7104d754a06370d81259,3fd66200f964a5207bec1ee3,False,111 Minna Gallery,37.787488,-122.399274,Art Gallery
7,4d4b7104d754a06370d81259,49f168eef964a520aa691fe3,False,Punch Line Comedy Club,37.795218,-122.399957,Comedy Club
8,4d4b7104d754a06370d81259,43d28d6ef964a520172e1fe3,False,The Gateway Theatre,37.796725,-122.399840,Indie Theater
9,4d4b7104d754a06370d81259,4ab16bb8f964a520676920e3,False,Variety Screening Room,37.789459,-122.401676,Movie Theater


At this point we know all open venues.

Let's show them on the map

In [186]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map of New York using latitude and longitude values
map_target = folium.Map(location=[target['latitude'], target['longtitude']], zoom_start=15)

colors_array = cm.rainbow(np.linspace(0, 1, len(CATEGORIES)+10))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to map
for lat, lng, name, categoryId, category in zip(target_venues['Venue Latitude'], target_venues['Venue Longitude'], target_venues['Venue'], target_venues['CategoryId'], target_venues['Venue Category']):
    label = '{}, {}'.format(name, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color=rainbow[2+CATEGORIES.index(categoryId)],
        fill=True,
        fill_color=rainbow[2+CATEGORIES.index(categoryId)],
        fill_opacity=0.7,
        parse_html=False).add_to(map_target)  

# add the target

label = '{}'.format(target['name'])
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
    [target['latitude'], target['longtitude']],
    radius=10,
    popup=label,
    color=rainbow[-2],
    fill=True,
    fill_color=rainbow[-2],
    fill_opacity=0.7,
    parse_html=False).add_to(map_target) 
    
map_target